In [ ]:
!pip install pyspark
from pyspark.sql import *

spark= SparkSession.builder\
.master("local")\
.appName("Colab")\
.config('spark.ui.port', '4050')\
.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=dc8ddab82056ef247dde09f41925632ad06720a53e4ae6191f8733c5bf9f0486
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
from  pyspark import pandas

In [ ]:
import pandas as pd
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/phamtheds/predict-flight-delays?select=Predict_Flight_Delays_2022_M1_M4.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alialnujaidi
Your Kaggle Key: ··········


100%|██████████| 842M/842M [00:10<00:00, 82.4MB/s]


alialnujaidi

3b1e0a628c830e978368ab04b89840b1

In [ ]:
df = spark.read.option("header",True) \
     .csv("predict-flight-delays/Predict_Flight_Delays_2022_M1_M4.csv")

In [ ]:
df.select('DEP_TIME','DEP_DELAY','AIR_TIME','DISTANCE').show()

+--------+---------+--------+--------+
|DEP_TIME|DEP_DELAY|AIR_TIME|DISTANCE|
+--------+---------+--------+--------+
|  1221.0|     -3.0|    63.0|   323.0|
|  1214.0|    -10.0|    50.0|   323.0|
|  1218.0|     -6.0|    53.0|   323.0|
|  1217.0|     -7.0|    56.0|   323.0|
|  1218.0|     -6.0|    48.0|   323.0|
|  1219.0|     -5.0|    49.0|   323.0|
|  1217.0|     -7.0|    58.0|   323.0|
|  1220.0|     -4.0|    49.0|   323.0|
|  1220.0|     -4.0|    52.0|   323.0|
|  1223.0|     -1.0|    49.0|   323.0|
|  1218.0|     -6.0|    53.0|   323.0|
|  1224.0|      0.0|    59.0|   323.0|
|  1224.0|      0.0|    57.0|   323.0|
|  1213.0|    -11.0|    52.0|   323.0|
|  1217.0|     -7.0|    60.0|   323.0|
|  1222.0|     -2.0|    56.0|   323.0|
|  1220.0|     -4.0|    50.0|   323.0|
|  1223.0|     -1.0|    59.0|   323.0|
|  1008.0|     -2.0|    64.0|   323.0|
|  1007.0|     -3.0|    59.0|   323.0|
+--------+---------+--------+--------+
only showing top 20 rows



**`PreProcessing`**:

---



In [ ]:
df = df.withColumn("DEP_TIME",col("DEP_TIME").cast("double")).dropna("any")
df = df.withColumn("DEP_DELAY",col("DEP_DELAY").cast("double")).dropna("any")
df = df.withColumn("AIR_TIME",col("AIR_TIME").cast("double")).dropna("any")
df = df.withColumn("DISTANCE",col("DISTANCE").cast("double")).dropna("any")

In [ ]:
import pandas as pd

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
input_cols = ['DEP_TIME','DISTANCE','AIR_TIME']
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')
df = assembler.transform(df)

In [ ]:
df = df.select("features","DEP_DELAY")

In [ ]:
(train_data, test_data) = df.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
reg = LinearRegression(labelCol = "DEP_DELAY")

In [ ]:
model = reg.fit(train_data)

In [ ]:
res = model.evaluate(test_data)

In [ ]:
unlabeled_data = test_data.select("features")

In [ ]:
predictions = model.transform(unlabeled_data)

In [ ]:
predictions.show(5)

+-----------------+-------------------+
|         features|         prediction|
+-----------------+-------------------+
| [1.0,150.0,32.0]|-5.2567932199345355|
| [1.0,152.0,28.0]| -5.278999231410023|
| [1.0,351.0,53.0]| -4.843497116322919|
| [1.0,391.0,58.0]|  -4.75611611217702|
|[1.0,888.0,108.0]| -3.746224223406286|
+-----------------+-------------------+
only showing top 5 rows



In [ ]:
print("MAE Reg model: ", res.meanAbsoluteError)
print("MSE Reg model: ", res.meanSquaredError)
print("RMSE Reg model: ", res.rootMeanSquaredError)
print("R2 Reg model:", res.r2)
print("Adj R2 Reg model:", res.r2adj)

MAE Reg model:  22.243838985084892
MSE Reg model:  2637.0922212583196
RMSE Reg model:  51.35262623526006
R2 Reg model: 0.013693216594809154
Adj R2 Reg model: 0.013688456013371542
